In [7]:
import numpy as np
import pandas as pd

In [8]:
traindata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TRAIN.txt')#reading the data
testdata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TEST.txt')


/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#CANTIDAD DE CADA VALOR. VEMOS QUE TENEMOS VALORES ENTEROS, CON x, QUE EMPIEZAN POR 0, REALES, ...
val_count=traindata['Socio_Demo_01'].value_counts()
val_count

09991     103393
09992      27266
09994      21070
0X301      15867
09993      15664
09511      13291
05220       8631
06110       8172
09512       7183
06120       5219
07121       4637
04500       4380
05120       3621
08432       2922
06300       2626
09210       2617
9992.0      2597
9991.0      2502
04111       2496
04309       2423
09602       2356
02230       1979
09700       1961
02240       1820
03613       1733
05000       1669
03611       1550
9512.0      1485
9994.0      1427
02121       1365
           ...  
2462.0         1
5823.0         1
3831           1
4446           1
2415.0         1
4445           1
9812           1
9542.0         1
4423.0         1
2630           1
2640.0         1
8132           1
02464          1
8141           1
3403.0         1
2434.0         1
3523           1
3531.0         1
3533           1
3539           1
3621           1
3622           1
2653           1
5893           1
5892.0         1
2154           1
2156           1
2157          

In [10]:
#ESTO DE MOMENTO IGNORADLO. SE ME HA OLVIDADO COMENTARLO, EL VIERNES LO DIGO

#grouped_by_socio=traindata.groupby(['Socio_Demo_01'])['Poder_Adquisitivo'].mean().sort_values()

In [11]:
#CONVERTIMOS TODOS LOS VALORES EN NUMEROS ENTEROS

mapping={}
for i in range(val_count.shape[0]):
    mapping[val_count.index[i]]=i
    
traindata['Socio_Demo_01']=traindata['Socio_Demo_01'].map(mapping)
testdata['Socio_Demo_01']=testdata['Socio_Demo_01'].map(mapping)


In [12]:
#ESTO LO VAMOS A NECESITAR AHORA DESPUES PARA LOS VALORES FALTANTES

#OBTENEMOS LA FRECUENCIA RELATIVA DE CADA VALOR
val_count=traindata['Socio_Demo_01'].value_counts()/traindata.shape[0]
#A LA QUE TIENE MAS FRECUENCIA LE AÑADIMOS LO QUE FALTA PARA LLEGAR A 1(EN LA DIVISION SE PIERDEN DECIMALES Y FALTABA 0.0001 O ASI)
val_count[0]+=1-val_count.values.sum()

In [13]:
#RELLENAMOS LOS VALORES FALTANTES DE FORMA ALEATORIA PONDERADA POR LAS FRECUENCIAS RELATIVAS
#ES DECIR, CADA VALOR FALTANTE ES MAS PROBABLE QUE SE RELLENE CON LOS MAS FRECUENTES, ASI SE MANTIENE LA DISTRIBUCION DE PROBABILIDAD ORIGINAL INTACTA

n_socio_nulls=traindata['Socio_Demo_01'].isnull().sum()
traindata.loc[traindata['Socio_Demo_01'].isnull(),'Socio_Demo_01']=np.random.choice(val_count.index,n_socio_nulls,p=val_count.values)

n_socio_nulls=testdata['Socio_Demo_01'].isnull().sum()
testdata.loc[testdata['Socio_Demo_01'].isnull(),'Socio_Demo_01']=np.random.choice(val_count.index,n_socio_nulls,p=val_count.values)

#YA NO FALTAN
traindata['Socio_Demo_01'].isnull().sum(),testdata['Socio_Demo_01'].isnull().sum()

(0, 0)

In [14]:
#ESTABA TODAVIA EN FLOAT, LO PASAMOS A INT
traindata['Socio_Demo_01']=traindata['Socio_Demo_01'].astype('int')
testdata['Socio_Demo_01']=testdata['Socio_Demo_01'].astype('int')

traindata['Socio_Demo_01'].value_counts()

0      103565
1       27310
2       21112
3       15893
4       15698
5       13311
6        8651
7        8191
8        7200
9        5227
10       4641
11       4383
12       3627
13       2927
14       2634
15       2619
16       2604
17       2512
18       2500
19       2427
20       2361
21       1982
22       1964
23       1821
24       1736
25       1671
26       1554
27       1488
28       1429
29       1366
        ...  
907         1
908         1
914         1
909         1
913         1
910         1
898         1
896         1
875         1
895         1
876         1
877         1
878         1
879         1
880         1
912         1
882         1
883         1
884         1
885         1
886         1
887         1
888         1
889         1
890         1
891         1
892         1
893         1
894         1
899         1
Name: Socio_Demo_01, Length: 921, dtype: int64

In [15]:
#LA CORRELACION DE LA VARIABLE CON LA OBJETIVO (ESTO ES BASTANTE INUTIL DADO QUE ES UNA VARIABLE NOMINAL)
#LO COMENTO
#traindata[['Socio_Demo_01','Poder_Adquisitivo']].corr()

In [16]:
#GUARDO LOS DATOS LIMPIOS
traindata.to_csv('train.csv',sep=',')
testdata.to_csv('test.csv',sep=',')